<a href="https://colab.research.google.com/github/muffaddal52/Data-Analytics-using-AI/blob/main/Unlock_Actionable_Insights_from_Surveys_using%C2%A0AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib certifi pip-system-certs

# Read Data From Google Spreadsheets using Python

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a your survey spreadsheet.
SAMPLE_SPREADSHEET_ID = "1K6PZoYMyChogoQZDsLXh1YWA3ROTf44kxjc67McvrFM"
SAMPLE_RANGE_NAME = "Sheet1"

import pandas as pd
def read_spreadsheet():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    # print("Name, Major:")
    # for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      # print(f"{row[0]}, {row[4]}")
    df = pd.DataFrame(values[1:], columns=values[0])
    return df
  except HttpError as err:
    print(err)

df = read_spreadsheet()
df.head(6)

# if __name__ == "__main__":
  # read_spreadsheet()

# Create AI Agent Using Langchain

In [ ]:
import langchain
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType

import pandas as pd

open_api_key = "insert open ai key here"
#provide context to our Agent
prefix = """
You are working with 1 pandas dataframes in Python named df1.

[add the details about the survey here to provide context to the AI]

you should use the tools below to answer the question posed of you:
"""

In [ ]:
open_api_key = <your open ai key here>
model ="gpt-4-1106-preview"
chat = ChatOpenAI(openai_api_key = open_api_key,model = model , temperature=0.0)
agent = create_pandas_dataframe_agent(chat, df, PREFIX = prefix, verbose = True, max_iterations=10, handle_parsing_errors=True )


# Generate Insights

## Get Executive Summary

In [ ]:
agent.run("""write an executive summary that succinctly captures the key information about to the questions answered by users. Ensure the summary is informative, and provides a comprehensive overview.
                        Consider both the questions and answers to generate the summary.""")

## Get Questions Insights

In [ ]:
# get the list of questions as text
result = agent.run("give me list of questions in the data set. I need only the questions as a list nothing else. Don't even add any other text just the list of questions")

# convert string of questions into list
import ast
questions_list = ast.literal_eval(result)

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
for question in questions_list:
    result = agent.run(f""" You are an expert growth marketing manager. Extract an important insights from the question below, you need to only consider and filter the dataset for this question. Use % to show distribution of answers as well. Find any insights or suggestions for improvements based on people responses.
        {question}""")
    printmd("### For Question: "+ question)
    printmd(result)
